## Module 9: Control creation of feature groups using IAM policies



Tagging is a useful technique for adopting cloud management and governance techniques. However, it can be difficult to ensure proper tagging behavior if it isn't enforced through proper mechanisms. To meet security and compliance requirements, you may need fine-grained control over the tags that are applied to create a feature store. It is critical for customers that are required to audit access to feature data and ensure the right level of security is in place. In this notebook, we have achieved  this using AWS Identity and Access Management (IAM) policies.In our example, we assume the customer has identified some tags that must be present to help ensure proper security and governance. We show a way to enforce such a requirement.

Identity-based policies decide whether someone can create, access, or delete AWS resources in an AWS account. By specifying the `RequestTag` condition key in the IAM policy, you can control access to Sagemaker API calls. For example, you require that a feature store created by an IAM user have a tag with the key environment and values `dev, production, or staging`. You will create an IAM policy with a `RequestTag` condition that denies access to the CreateFeatureGroup action until the tag value is in dev, production, or staging and attach it to the Sagemaker execution role. Now if an IAM user tries to create a feature store for customer data without any tags or tags other than expected values, the request will get denied. 


![FS_Diagram](../../images/fs_security_iam_policy_governance_tag.png)

#### Imports

In [ ]:
import sagemaker
import boto3
import sys
import pandas as pd
import numpy as np
import io
import time
from time import gmtime, strftime, sleep
import json
import logging
from sagemaker.session import Session
from sagemaker import get_execution_role

In [ ]:
sm_version = sagemaker.__version__
major, minor, patch = sm_version.split('.')
if int(major) < 2 or int(minor) < 125:
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'sagemaker==2.125.0'])
    importlib.reload(sagemaker)

In [ ]:
logger = logging.getLogger('__name__')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

#### Initialize default parameters


In [ ]:
prefix = 'sagemaker-feature-store'
role = get_execution_role()

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
s3_bucket_name = sagemaker_session.default_bucket()

#### Load and explore dataset
In this notebook, we will be using the dataset created in Module 1/Prepare datasets of the workshop.

In [ ]:
customer_data = pd.read_csv('../.././data/transformed/customers.csv')

In [ ]:
customer_data.head()

### IAM policy update
Add the below policy to the IAM role (SageMaker Execution Role) used for this notebook as inline policy. It will allow you to modify the IAM policy programatically in the following sections.
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "iam:CreatePolicy",
                "iam:DetachRolePolicy",
                "iam:ListAttachedRolePolicies",
                "iam:DeletePolicy",
                "iam:AttachRolePolicy"
            ],
            "Resource": "*"
        }
    ]
}
```

### Create and add IAM policies based on tags to the existing role


Define IAM policies to restrict feature store creation without specific tags

In [ ]:
"""creation of iam policy to restrict feature store creation without specific tags."""
    
_iam_tag_policy = {

                        "Version": "2012-10-17",
                        "Statement": [
                            {
                                "Effect": "Deny",
                                "Action": "sagemaker:CreateFeatureGroup",
                                "Resource": "arn:aws:sagemaker:*:*:feature-group/*",
                                "Condition": {
                                    "StringNotEquals": {
                                        "aws:RequestTag/environment": [
                                            "dev",
                                            "production",
                                            "staging"
                                        ]
                                    }
                                }
                            }
                        ]
                    }

Create IAM policies and attach them to the IAM role (SageMaker Execution Role)

In [ ]:
# Attach IAM policy to restrict SageMaker Execution Role
timestamp = int(time.time())
iam_client = boto3.client('iam')
role_name = role.split('/')[-1] # get the role name from role arn

policy_res = iam_client.create_policy(
    PolicyName=f'Amazon_SageMaker_Tag_Policy_{timestamp}',
    PolicyDocument=json.dumps(_iam_tag_policy)
)
policy_arn = policy_res['Policy']['Arn']

policy_attach_res = iam_client.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)
## IAM is eventually consistent so added sleep to allow it to propogate to other regions
time.sleep(120) 

Create a feature group


In [ ]:
customers_feature_group_name = "customers-feature-group-" + strftime("%d-%H-%M-%S", gmtime())

In [ ]:
#Instantiate a FeatureGroup object for customers data_data
from sagemaker.feature_store.feature_group import FeatureGroup

customers_feature_group = FeatureGroup(
    name=customers_feature_group_name, sagemaker_session=sagemaker_session
)

In [ ]:
current_time_sec = int(round(time.time()))
record_identifier_feature_name = "customer_id"

In [ ]:
#Append EventTime feature to your data frame. This parameter is required, and time stamps each data point
customer_data["EventTime"] = pd.Series([current_time_sec] * len(customer_data), dtype="float64")

In [ ]:
#Load feature definitions to your feature group.
customers_feature_group.load_feature_definitions(data_frame=customer_data)

### 1. Test Deny when creating a feature store without a tag

You should receive an error message in this scenario. `An error occurred (AccessDeniedException) when calling the CreateFeatureGroup operation: User: arn:aws:sts::*ACCOUNT-ID*:assumed-role/AmazonSageMaker-ExecutionRole-*ID*/SageMaker is not authorized to perform: sagemaker:CreateFeatureGroup on resource: arn:aws:sagemaker:*REGION*:*ACCOUNT-ID*:feature-group/customers-feature-group-*TIMESTAMP* with an explicit deny in an identity-based policy`

In [ ]:
customers_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True
)

### 2. Test Deny when creating a feature store with an incorrect tag value

You should receive an error message in this scenario. `An error occurred (AccessDeniedException) when calling the CreateFeatureGroup operation: User: arn:aws:sts::*ACCOUNT-ID*:assumed-role/AmazonSageMaker-ExecutionRole-*ID*/SageMaker is not authorized to perform: sagemaker:CreateFeatureGroup on resource: arn:aws:sagemaker:*REGION*:*ACCOUNT-ID*:feature-group/customers-feature-group-*TIMESTAMP* with an explicit deny in an identity-based policy`

In [ ]:
customers_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
    tags=[{'Key':'environment','Value':'qa'}]
)

### 3. Test allow when creating a feature store with a allowed tag value

In [ ]:
customers_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
    tags=[{'Key':'environment','Value':'staging'}]
)

### Clean up

First delete the tag based granular access IAM policy 

In [ ]:
# Detach iam policy added to the SageMaker execution role
policy_detach_res = iam_client.detach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_arn
)

# Delete the IAM policy
delete_policy_res = iam_client.delete_policy(
    PolicyArn=policy_arn
)

Then delete the feature store

In [ ]:
feature_group_name = customers_feature_group.describe()['FeatureGroupName']
index = 1
while index < 10:
    if customers_feature_group.describe()['FeatureGroupStatus'] == 'Created':
       customers_feature_group.delete()
       print(f"{feature_group_name} deleted.")
       index = 10
    elif customers_feature_group.describe()['FeatureGroupStatus'] == 'Creating':
        print(f"{feature_group_name} is still in Creating status. Will try delete operation after 10 seconds.")
        time.sleep(10)
        index = index + 1

Finally delete the inline policy you added to the IAM role (SageMaker Execution Role) earlier in this notebook.
```
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "VisualEditor0",
            "Effect": "Allow",
            "Action": [
                "iam:CreatePolicy",
                "iam:DetachRolePolicy",
                "iam:ListAttachedRolePolicies",
                "iam:DeletePolicy",
                "iam:AttachRolePolicy"
            ],
            "Resource": "*"
        }
    ]
}
```